In [91]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings('ignore')

In [92]:
df = pd.read_csv("cleaned_post_comment.csv")

In [93]:
df=df[:15000]

In [96]:
df.shape

(15000, 3)

In [97]:
df.head()

,Unnamed: 0,title_body,comment
0,0,[AITA] I wrote an explanation in TIL and came ...,<|response|> There is nothing wrong with your ...
1,1,[AITA] Threw my parent's donuts away My parent...,<|response|> Yep. You're the asshole. If you w...
2,2,I told a goth girl she looked like a clown. I ...,<|response|> Kids say ridiculous things all th...
3,3,[AItA]: Argument I had with another redditor i...,<|response|> I'm not reading that. But don't s...
4,4,[AITA] I let my story get a little long and bo...,<|response|> A real yawn? Or an I'm-trying-to-...


In [98]:
df

,Unnamed: 0,title_body,comment
0,0,[AITA] I wrote an explanation in TIL and came ...,<|response|> There is nothing wrong with your ...
1,1,[AITA] Threw my parent's donuts away My parent...,<|response|> Yep. You're the asshole. If you w...
2,2,I told a goth girl she looked like a clown. I ...,<|response|> Kids say ridiculous things all th...
3,3,[AItA]: Argument I had with another redditor i...,<|response|> I'm not reading that. But don't s...
4,4,[AITA] I let my story get a little long and bo...,<|response|> A real yawn? Or an I'm-trying-to-...
...,...,...,...
14995,14995,AITA for telling people about my breakup So a ...,<|response|> Nta. She's obviously trying to ma...
14996,14996,WIBTA for wanting to leave my girlfriend and t...,<|response|> >I want to tell her that I don’t ...
14997,14997,AITA for for spending less and making home mad...,<|response|> NTA gifts aren't about how much m...
14998,14998,AITA for telling my uncle how much of a deadbe...,<|response|> NTA - It's not your job to cover ...


In [99]:
df["label"] = ""

In [100]:
df.head()

,Unnamed: 0,title_body,comment,label
0,0,[AITA] I wrote an explanation in TIL and came ...,<|response|> There is nothing wrong with your ...,
1,1,[AITA] Threw my parent's donuts away My parent...,<|response|> Yep. You're the asshole. If you w...,
2,2,I told a goth girl she looked like a clown. I ...,<|response|> Kids say ridiculous things all th...,
3,3,[AItA]: Argument I had with another redditor i...,<|response|> I'm not reading that. But don't s...,
4,4,[AITA] I let my story get a little long and bo...,<|response|> A real yawn? Or an I'm-trying-to-...,


In [101]:
from tqdm import tqdm

def Search_Label_AITA():
    for i in tqdm(range(len(df['comment']))):
        if "NTA" in df['comment'][i]:
            df["label"][i] = "NTA"
        elif "YTA" in df['comment'][i]:
            df["label"][i] = "YTA"
        elif "YWBTA" in df['comment'][i]:
            df["label"][i] = "YWBTA"
        elif "YWNBTA" in df['comment'][i]:
            df["label"][i] = "YWNBTA"
        elif "ESH" in df['comment'][i]:
            df["label"][i] = "ESH"
        elif "NAH" in df['comment'][i]:
            df["label"][i] = "NAH"
        elif "INFO" in df['comment'][i]:
            df["label"][i] = "INFO"

In [102]:
Search_Label_AITA()

100%|█████████████████████████████████████████████████████████████████████████| 15000/15000 [00:01<00:00, 11296.20it/s]


In [103]:
df = df.drop(df[df['label'] == ''].index)

In [104]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [105]:
df.shape

(4646, 3)

In [106]:
class_distribution = df['label'].value_counts(normalize=True)

print("Class Distribution:")
print(class_distribution)

Class Distribution:
label
NTA       0.744296
YTA       0.181231
NAH       0.040680
ESH       0.023676
INFO      0.006027
YWBTA     0.003659
YWNBTA    0.000430
Name: proportion, dtype: float64


In [14]:
df.shape

(4646, 3)

In [15]:
df.columns

Index(['title_body', 'comment', 'label'], dtype='object')

In [16]:
# Convert text data to a sparse matrix
vectorizer = TfidfVectorizer()
sparse_matrix = vectorizer.fit_transform(df['title_body'])

In [17]:
dense_matrix = sparse_matrix.toarray()

In [18]:
dense_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

X = dense_matrix
y = df['label']
# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
X_test.shape

(930, 21326)

In [21]:
y_train

14912    NAH
2994     NTA
13122    NTA
13128    NTA
14617    YTA
        ... 
14751    ESH
4715     NTA
13304    NTA
14052    NTA
5411     NTA
Name: label, Length: 3716, dtype: object

In [22]:
# Create a Gaussian Naive Bayes classifier
clf = GaussianNB()


# Train the classifier
clf.fit(X_train, y_train)

GaussianNB()

In [23]:
# Make predictions on the test set
y_pred = clf.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.7043010752688172


In [71]:
X_test.shape

(930, 21326)

In [62]:
user_input = 'AITA for hating my dad and uncle? I\'m using a throw away. So, I am still in High School, but growing up my life hasn\'t been very fun. My parents divorced early and I lived with my mom from 6-13 in Ohio, whereas I had been in Texas before. Anyway, When I was 13, I came home from school one day and my mom had been arrested for like the 5th time for doing drugs. But this time was permanent. (I haven\'t seen her since.) After that, my grandma drove up to Ohio from Texas to get me, and brought my dad with her. Then we drove back, and I moved in with my grandma. My grandma is awesome. Anyway, I found out shortly after moving in that my uncle would also be living with my grandma. My uncle is 40. He has "mental problems" as my grandparents describe it, so they use that as an excuse to put up with his bs, such as leaving messes everywhere, trashes my grandmas car, never having a job, and they said he will probably never move out because is isn\'t capable. So that went on for about 2 years, and oh man I hated it. His blatant disrespect for how lucky he was made me so angry. He was practically sucking financial resources out of my grandparents for free because he was lazy. Two years of rent free life. That combined with the fact that he is an enormous jerk really makes me hate him. After those 2 years, my grandma brings up that my dad, back in Dallas (I live in a very small town about an hour away) has been living on the streets long enough, so she lets HIM move in as well. When HE moves in, he does the same thing as my uncle. Freeloading, whining, being ungrateful, and just generally being a drain and a pain in the ass. Having to live with these two constantly makes me dread being home. It has made me really sour. I literally never smile when they are around, and I never speak to them. When they speak to me I just answer with quick one word answers and try to end the conversation as quickly as possible. My uncle still lives here, going on nearly 4 years (I\'m 16.). My dad has been here for about 3 months, but he is no closer to moving out than he was when he got here. I am constantly reassured he will be gone as soon as he gets his first paycheck, but he doesn\'t have a job and I\'m not holding my breathe. My grandma knows I am super unhappy with them being here, but she and my sister both say I am being a dick. They say that my dad and uncle are family, and I shouldn\'t hate them, that I should give them more chances. Since my entire life all of my family, besides my little sister and grandparents, have been drug addicts, failures, and freeloaders, I never really learned to value'
userlist = []
userlist.append(user_input)

In [63]:
userlist

['AITA for hating my dad and uncle? I\'m using a throw away. So, I am still in High School, but growing up my life hasn\'t been very fun. My parents divorced early and I lived with my mom from 6-13 in Ohio, whereas I had been in Texas before. Anyway, When I was 13, I came home from school one day and my mom had been arrested for like the 5th time for doing drugs. But this time was permanent. (I haven\'t seen her since.) After that, my grandma drove up to Ohio from Texas to get me, and brought my dad with her. Then we drove back, and I moved in with my grandma. My grandma is awesome. Anyway, I found out shortly after moving in that my uncle would also be living with my grandma. My uncle is 40. He has "mental problems" as my grandparents describe it, so they use that as an excuse to put up with his bs, such as leaving messes everywhere, trashes my grandmas car, never having a job, and they said he will probably never move out because is isn\'t capable. So that went on for about 2 years, 

In [64]:
new_df = pd.DataFrame(userlist)

In [65]:
new_df[0][0]

'AITA for hating my dad and uncle? I\'m using a throw away. So, I am still in High School, but growing up my life hasn\'t been very fun. My parents divorced early and I lived with my mom from 6-13 in Ohio, whereas I had been in Texas before. Anyway, When I was 13, I came home from school one day and my mom had been arrested for like the 5th time for doing drugs. But this time was permanent. (I haven\'t seen her since.) After that, my grandma drove up to Ohio from Texas to get me, and brought my dad with her. Then we drove back, and I moved in with my grandma. My grandma is awesome. Anyway, I found out shortly after moving in that my uncle would also be living with my grandma. My uncle is 40. He has "mental problems" as my grandparents describe it, so they use that as an excuse to put up with his bs, such as leaving messes everywhere, trashes my grandmas car, never having a job, and they said he will probably never move out because is isn\'t capable. So that went on for about 2 years, a

In [66]:
df['title_body'][98]

'AITA for hating my dad and uncle? I\'m using a throw away. So, I am still in High School, but growing up my life hasn\'t been very fun. My parents divorced early and I lived with my mom from 6-13 in Ohio, whereas I had been in Texas before. Anyway, When I was 13, I came home from school one day and my mom had been arrested for like the 5th time for doing drugs. But this time was permanent. (I haven\'t seen her since.) After that, my grandma drove up to Ohio from Texas to get me, and brought my dad with her. Then we drove back, and I moved in with my grandma. My grandma is awesome. Anyway, I found out shortly after moving in that my uncle would also be living with my grandma. My uncle is 40. He has "mental problems" as my grandparents describe it, so they use that as an excuse to put up with his bs, such as leaving messes everywhere, trashes my grandmas car, never having a job, and they said he will probably never move out because is isn\'t capable. So that went on for about 2 years, a

In [67]:
user_sparse = vectorizer.fit_transform(new_df[0]).toarray()


In [68]:
user_sparse

array([[0.03902114, 0.01951057, 0.01951057, 0.01951057, 0.05853171,
        0.01951057, 0.05853171, 0.01951057, 0.01951057, 0.01951057,
        0.07804228, 0.05853171, 0.33167969, 0.01951057, 0.01951057,
        0.01951057, 0.03902114, 0.03902114, 0.01951057, 0.01951057,
        0.17559513, 0.01951057, 0.03902114, 0.01951057, 0.03902114,
        0.03902114, 0.03902114, 0.11706342, 0.01951057, 0.09755285,
        0.01951057, 0.01951057, 0.01951057, 0.01951057, 0.01951057,
        0.01951057, 0.01951057, 0.09755285, 0.01951057, 0.01951057,
        0.01951057, 0.01951057, 0.01951057, 0.01951057, 0.03902114,
        0.01951057, 0.09755285, 0.01951057, 0.01951057, 0.01951057,
        0.01951057, 0.01951057, 0.01951057, 0.01951057, 0.01951057,
        0.01951057, 0.01951057, 0.01951057, 0.03902114, 0.01951057,
        0.01951057, 0.01951057, 0.01951057, 0.01951057, 0.01951057,
        0.01951057, 0.01951057, 0.01951057, 0.01951057, 0.01951057,
        0.03902114, 0.01951057, 0.01951057, 0.13

In [69]:
y_predfff = clf.predict(user_sparse)

ValueError: X has 231 features, but GaussianNB is expecting 21326 features as input.

In [123]:
import pandas as pd

# Assuming df is your DataFrame
temp = []
new_df = df[:1000]
for index, row in new_df.iterrows():
    temp.append((row['title_body'], row['label']))

# Print the first few elements of the list for verification
print(temp[:5])


[('AITA for hating my dad and uncle? I\'m using a throw away. So, I am still in High School, but growing up my life hasn\'t been very fun. My parents divorced early and I lived with my mom from 6-13 in Ohio, whereas I had been in Texas before. Anyway, When I was 13, I came home from school one day and my mom had been arrested for like the 5th time for doing drugs. But this time was permanent. (I haven\'t seen her since.) After that, my grandma drove up to Ohio from Texas to get me, and brought my dad with her. Then we drove back, and I moved in with my grandma. My grandma is awesome. Anyway, I found out shortly after moving in that my uncle would also be living with my grandma. My uncle is 40. He has "mental problems" as my grandparents describe it, so they use that as an excuse to put up with his bs, such as leaving messes everywhere, trashes my grandmas car, never having a job, and they said he will probably never move out because is isn\'t capable. So that went on for about 2 years,

In [124]:
temp[10]

('AITA: I don’t want friends to invite their friends to my house. Having a small bbq with a few friends this weekend, and bc of circumstances we had to invite someone’s brother and he is now asking if he can invite more people. We know the people he wants to invite but we don’t want them to come... 1. The husband is super obnoxious and complains about everything 2. They never leave/party way harder than we do. I’m not really sure what to tell him bc we really don’t want these people to come but I feel like an asshole saying no.',
 'NTA')

In [125]:
len(temp)

1000

In [126]:
new_df.shape

(1000, 3)

In [127]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Sample dataset (replace this with your dataset)
data = temp

# Preprocess data
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalnum()]
    tokens = [token for token in tokens if token not in stop_words]
    return tokens

# Create feature set
all_words = FreqDist()
for text, _ in data:
    for word in preprocess_text(text):
        all_words[word] += 1

word_features = list(all_words.keys())

def extract_features(text):
    words = set(preprocess_text(text))
    features = {}
    for word in word_features:
        features[word] = (word in words)
    return features

# Split data into training and test sets
featuresets = [(extract_features(text), label) for text, label in data]
train_set, test_set = train_test_split(featuresets, test_size=0.3, random_state=42)

# Train the Naive Bayes classifier
classifier = nltk.NaiveBayesClassifier.train(train_set)

# Evaluate the classifier
y_true = [label for _, label in test_set]
y_pred = [classifier.classify(features) for features, _ in test_set]
accuracy = accuracy_score(y_true, y_pred)

print(f"Accuracy: {accuracy:.2f}")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\akhas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\akhas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\akhas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Accuracy: 0.78


In [132]:
# Get input from user
user_input = input("Enter your scenario: ")

# Preprocess user input
user_features = extract_features(user_input)

# Use the trained classifier to predict the label
prediction = classifier.classify(user_features)

# Display the prediction
print(f"Based on your scenario, you are considered: {prediction}")

Enter your scenario:  meow meow nigga


Based on your scenario, you are considered: NTA


In [128]:
y_true

['YTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'YTA',
 'NTA',
 'NTA',
 'YTA',
 'NTA',
 'NTA',
 'NTA',
 'YTA',
 'NTA',
 'NTA',
 'YTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'YTA',
 'NTA',
 'NTA',
 'YTA',
 'NTA',
 'YTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'YTA',
 'YTA',
 'NTA',
 'NTA',
 'YTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'YTA',
 'YTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'YTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'YTA',
 'YTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'YTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'YTA',
 'NTA',
 'NTA',
 'NTA',
 'YTA',
 'YTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'YTA',
 'NTA',
 'YTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',


In [129]:
y_pred

['NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'YTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'YTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'YTA',
 'NTA',
 'NTA',
 'YTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'YTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'YTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',
 'NTA',


In [130]:
userinput=[]
userinput.append("meow meow nigga")
ans = [classifier.classify(features) for features, _ in userinput]
print(ans)

ValueError: too many values to unpack (expected 2)